In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.20)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.68 sec - Loss 0.204868 - ACC 69.86% - ACC Mean 69.86% - AUC 75.27% - AUC Mean 75.27% - Deter 000
Ite 00050 - 1.07 sec - Loss 0.094655 - ACC 71.99% - ACC Mean 71.85% - AUC 76.10% - AUC Mean 76.82% - Deter 041
Ite 00100 - 1.02 sec - Loss 0.070284 - ACC 71.54% - ACC Mean 71.92% - AUC 76.24% - AUC Mean 77.84% - Deter 033
Ite 00117 - 0.88 sec - Loss 0.066089 - ACC 70.73% - ACC Mean 71.33% - AUC 75.89% - AUC Mean 76.98% - Deter 050
Early stop ite 117, rollback to correction of ite 67, whith acc of 73.16% and auc of 80.28%
OUR METHOD RUN 1/10 - acc: 73.16% +- 0.0% - auc: 80.28% +- 0.0%
Ite 00000 - 0.99 sec - Loss 0.215856 - ACC 72.32% - ACC Mean 72.32% - AUC 76.74% - AUC Mean 76.74% - Deter 000
Ite 00050 - 0.92 sec - Loss 0.109192 - ACC 72.01% - ACC Mean 71.99% - AUC 76.41% - AUC Mean 76.94% - Deter 033
Ite 00067 - 1.20 sec - Loss 0.094926 - ACC 73.26% - ACC Mean 71.78% - AUC 77.70% - AUC Mean 76.39% - Deter 050
Early stop ite 67, rollback to correction of ite 17, whith acc of 7

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 61.93% +- 0.0% - auc: 67.86% +- 0.0%
MEAN - acc: 73.73% +- 0.0% - auc: 78.42% +- 0.0%
KNN - acc: 67.62% +- 0.0% - auc: 73.0% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [01:03<00:00, 79.14it/s]



GAIN RUN 1/10 - acc: 70.31% +- 2.7% - auc: 75.08% +- 4.42%



100%|██████████| 5000/5000 [00:56<00:00, 88.48it/s]



GAIN RUN 2/10 - acc: 68.19% +- 1.93% - auc: 74.25% +- 1.47%



100%|██████████| 5000/5000 [00:53<00:00, 93.45it/s] 


...


GAIN RUN 8/10 - acc: 69.33% +- 2.1% - auc: 73.97% +- 1.63%



100%|██████████| 5000/5000 [01:05<00:00, 76.56it/s]



GAIN RUN 9/10 - acc: 69.59% +- 1.04% - auc: 74.62% +- 2.24%



100%|██████████| 5000/5000 [01:07<00:00, 74.20it/s]



GAIN RUN 10/10 - acc: 67.88% +- 2.93% - auc: 73.51% +- 3.36%

GAIN GLOBAL - acc: 69.97% +- 1.27% - auc: 75.15% +- 1.11%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 72.52% +- 0.0% - auc: 76.1% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 1/10 - acc: 71.88% +- 2.82% - auc: 76.98% +- 1.6%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 2/10 - acc: 71.37% +- 2.51% - auc: 76.27% +- 1.7%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 3/10 - acc: 71.15% +- 2.52% - auc: 76.32% +- 1.09%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 4/10 - acc: 70.48% +- 1.37% - auc: 76.18% +- 1.23%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 5/10 - acc: 71.52% +- 1.98% - auc: 77.13% +- 3.81%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 6/10 - acc: 72.42% +- 0.76% - auc: 77.17% +- 2.76%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 7/10 - acc: 71.72% +- 2.32% - auc: 77.44% +- 1.76%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iterat

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 72.22% +- 2.39% - auc: 75.92% +- 1.93%
SOFTIMPUTE RUN 2/10 - acc: 73.39% +- 1.63% - auc: 76.42% +- 1.95%
SOFTIMPUTE RUN 3/10 - acc: 73.74% +- 1.62% - auc: 76.98% +- 1.57%
SOFTIMPUTE RUN 4/10 - acc: 70.27% +- 2.39% - auc: 74.58% +- 2.61%
SOFTIMPUTE RUN 5/10 - acc: 70.29% +- 1.8% - auc: 74.99% +- 1.74%
SOFTIMPUTE RUN 6/10 - acc: 71.74% +- 2.45% - auc: 75.65% +- 3.32%
SOFTIMPUTE RUN 7/10 - acc: 73.13% +- 0.7% - auc: 76.83% +- 2.05%
SOFTIMPUTE RUN 8/10 - acc: 71.98% +- 2.18% - auc: 75.4% +- 2.84%
SOFTIMPUTE RUN 9/10 - acc: 72.11% +- 1.15% - auc: 76.5% +- 2.09%
SOFTIMPUTE RUN 10/10 - acc: 71.74% +- 2.13% - auc: 74.62% +- 2.23%

SOFTIMPUTE GLOBAL - acc: 72.06% +- 1.11% - auc: 75.79% +- 0.84%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.9800

SINKHORN RUN 1/10 - acc: 69.27% +- 1.17% - auc: 73.45% +- 2.86%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.9946

SINKHORN RUN 2/10 - acc: 69.62% +- 2.08% - auc: 74.6% +- 4.8%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.9565

SINKHORN RUN 3/10 - acc: 71.45% +- 1.0% - auc: 76.87% +- 2.6%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 6.1261

SINKHORN RUN 4/10 - acc: 72.13% +- 2.32% - auc: 76.36% +- 2.98%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 6.0931

SINKHORN RUN 5/10 - acc: 70.61% +- 1.67% - auc: 74.39% +- 2.95%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 6.1518

SINKHORN RUN 6/10 - acc: 71.24% +- 1.08% - auc: 76.5% +- 2.35%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 6.1586

SINKHORN RUN 7/10 - acc: 70.71% +- 2.62% - auc: 74.71% +- 1.7%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 6.3045

SINKHORN RUN 8/10 - acc: 70.86% +- 2.1% - auc: 75.2% +-

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 68.24% +- 3.96% - auc: 71.19% +- 2.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 69.15% +- 3.11% - auc: 76.61% +- 2.45%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 70.17% +- 1.99% - auc: 76.17% +- 1.58%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 69.8% +- 2.52% - auc: 74.29% +- 2.67%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 68.09% +- 3.15% - auc: 73.16% +- 2.07%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 70.12% +- 3.98% - auc: 74.14% +- 3.33%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 70.36% +- 1.94% - auc: 74.7% +- 2.28%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 70.17% +- 1.97% - auc: 76.87% +- 2.74%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly worse than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is even with MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bett